In [2]:
# # save_model.py
# import torch
# from models.experimental import attempt_load

# # Load .pt weights (YOLOv7)
# model = attempt_load('../runs/train/exp/weights/best.pt', map_location='cpu')
# model.eval()

# # Save to pickle (.pkl)
# torch.save(model, './chest_Xray_weight.pkl')
# print("Model saved to yolov7_model.pkl")

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
Model saved to yolov7_model.pkl


In [4]:
# inference_from_pkl.py
import torch
import cv2
import numpy as np
from utils.general import non_max_suppression, scale_coords
from utils.datasets import letterbox

# Load .pkl model
model = torch.load('../../Piethon_test/CXR_Yolo/yolov7/chest_Xray_weight.pkl', map_location='cpu', weights_only=False)
model.eval()

# Load image
img0 = cv2.imread('./0a2d01ecb9e01cf972c1e1d31ccacb98.png')  # 이미지 경로
assert img0 is not None

# Preprocess
img = letterbox(img0, new_shape=640)[0]  # Resize
img = img.transpose((2, 0, 1))[::-1]     # HWC to CHW, BGR to RGB
img = np.ascontiguousarray(img)
img = torch.from_numpy(img).float()
img /= 255.0
img = img.unsqueeze(0)  # Add batch dim

# Inference
with torch.no_grad():
    pred, _ = model(img)

# NMS
pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45)[0]

# 결과 출력
if pred is not None and len(pred):
    pred[:, :4] = scale_coords(img.shape[2:], pred[:, :4], img0.shape).round()
    for *xyxy, conf, cls in pred:
        print(int(cls), "confidence score: ", float(conf.cpu()), "coordinate:", [int(b.cpu()) for b in xyxy])


3 confidence score:  0.8860486149787903 coordinate: [383, 478, 800, 645]
0 confidence score:  0.604362964630127 coordinate: [508, 227, 621, 327]
11 confidence score:  0.32202625274658203 coordinate: [569, 110, 747, 149]


/home/smaller225/anaconda3/envs/biomedparse/lib/python3.9/site-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
